# Laboratorio de regresión logística

|                |   |
:----------------|---|
| **Nombre**     | Julia Hernández Cárdenas  |
| **Fecha**      | 06/10/2025  |
| **Expediente** | 739208  | 

In machine learning, Support Vector Machines (SVM) are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. It is mostly used in classification problems. In this algorithm, each data item is plotted as a point in p-dimensional space (where p is the number of features), with the value of each feature being the value of a particular coordinate. Then, classification is performed by finding the hyper-plane that best differentiates the two classes (or more if we have a multi class problem):

$$ f(x) = w^T \varphi(x) + b $$

where $\varphi: X \rightarrow F $ is a function that makes each input point $x$ correspond to a point in F, where F is a Hilbert space.

In addition to performing linear classification, SVMs can efficiently perform a non-linear classification, implicitly mapping their inputs into high-dimensional feature spaces (more specifically using the kernel trick, like the RBF funcion). 

[1]

OLS utilizes the squared residuals to fit the parameters. Large residuals caused by outliers may worsen the accuracy significantly.

Support Vectors use piecewise linear functions to counter this, in which a hyperparameter  $\epsilon$ called the margin lets errors that are less or equal to it be 0, and error larger than it be $e - \epsilon$. 

The problem to solve is:

\begin{split}
        \min_{w, b, \xi, \xi^*} \mathcal{P}_\epsilon(w, b, \xi) &= \frac{1}{2} w^T w + c \sum_{k=1}^{N} \xi_k \\
        \text{s.t. } & y_k [w^T \varphi(x_k) - b] \geq 1- \xi_k,\ \ k = 1, ..., N \\
        & \xi_k \geq 0,\ \ k = 1, ..., N
\end{split}


The most important question that arises when using a SVM is how to choose the correct hyperplane. Consider the following scenarios:

### Scenario 1

In this scenario there are three hyperplanes called A, B, and C. Now, the problem is to identify the hyperplane which best differentiates the stars and the circles.

<center><img src="https://media.geeksforgeeks.org/wp-content/uploads/SVM_21-2.png" alt="what image shows"></center>

In this case, hyperplane B separates the stars and the circle betters, hence it is the correct hyperplane.


### Scenario 2

Now take another scenario where all three hyperplanes are segregating classes well. The question that arises is how to choose the best hyperplane in this situation.

<center><img src="https://media.geeksforgeeks.org/wp-content/uploads/SVM_4-2.png" alt="what image shows"></center>

In such scenarios, we calculate the margin (which is the distance between nearest data point and the hyperplane). The hyperplane with the largest margin will be considered as the correct hyperplane to classify the dataset.

Here C has the largest margin. Hence, it is considered as the best hyperplane.


### Kernels
Knowing 
$$ w = \sum_{k=1}^{N} \alpha_k y_k \varphi(x_k) $$

And
$$ y_{pred} = w^T \varphi(x) + b $$

Then 
$$ y_{pred} = (\sum_{k=1}^{N} \alpha_k y_k \varphi(x_k))^T \varphi(x) + b $$

Where $\varphi$ is a function that makes each input in $x$ correspond to a point in $F$ (a Hilbert space). This can be seen as processing and transforming the input featuers to keep the model's convexity. [2]

This also allows us to transform the inputs into another space where they might be more easily classified.

<center><img src=https://miro.medium.com/max/838/1*gXvhD4IomaC9Jb37tzDUVg.png alt="what image shows"></center>

## ROC and AUC

A ROC (Receiver Operating Characteristic) is a graph that shows how the classification model performs at the classification thresholds. 

ROC curves typically feature true positive rate on the Y axis, and false positive rate on the X axis. This means that the top left corner of the plot is the “ideal” point - a false positive rate of zero, and a true positive rate of one. This is not very realistic, but it does mean that a larger area under the curve (AUC) is usually better. [3]

True Positive Rate is a synonym for Recall and defined as:
$$ TPR = \frac{TP}{TP + FN} $$

False Positive Rate is a synonym for Specificity and defined as:

$$ FPR = \frac{FP}{FP + TN} $$

ROC curves are typically used in binary classification to study the output of a classifier. In order to extend ROC curve and ROC area to multi-label classification, it is necessary to binarize the output. One ROC curve can be drawn per label, but one can also draw a ROC curve by considering each element of the label indicator matrix as a binary prediction (micro-averaging).

E.g. If you lower a classification threshold, more items would be classified as positive, increasing False Positives and True Positives.

AUC stands for Area under the ROC.

## Ejercicio 1

- Utiliza el dataset `Iris`, modela con SVC y haz Cross-Validation de diferentes kernels ('linear', 'poly', 'rbf', 'sigmoid').
- Modela con LogisticRegression.
- El método de Cross-Validation es K-Folds con $k=10$.
- Utiliza el AUC como métrico de Cross-Validation.
- Compara resultados.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import t 
from sklearn.svm import SVC
from sklearn.datasets import load_iris 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, train_test_split, cross_validate
from sklearn.compose import ColumnTransformer  

In [2]:
df = load_iris()
print("Dataset Shape:", df.data.shape, df.target.shape)
print("Dataset Features:", df.feature_names)
print("Dataset Target:", df.target_names)
X = df.data
y = df.target

Dataset Shape: (150, 4) (150,)
Dataset Features: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Dataset Target: ['setosa' 'versicolor' 'virginica']


In [3]:
indice = list(range(X.shape[1]))
preprocessor = ColumnTransformer(
    transformers=[('num', StandardScaler(), indice)],
    remainder='drop'
)

kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

#AUC
scoring = {
    'accuracy': 'accuracy',
    'auc_ovr': 'roc_auc_ovr'
}
resultados = []

kernels = ['linear', 'poly', 'rbf', 'sigmoid']
for k in kernels:
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', SVC(kernel=k, gamma='scale', degree=2, probability=True))
    ])
    
    scores = cross_validate(pipeline, X, y, cv=kfolds, scoring=scoring, n_jobs=-1)
    resultados.append({
        'modelo': f'svc_{k}',
        'modelo_mean': np.mean(scores['test_accuracy']),
        'modelo_std':  np.std(scores['test_accuracy']),
        'AUC_mean': np.mean(scores['test_auc_ovr']),
    })

# Logistic Regression
pipe_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])

scores_lr = cross_validate(pipe_lr, X, y, cv=kfolds, scoring=scoring, n_jobs=-1)

resultados.append({
    'modelo': 'LogisticReg',
    'modelo_mean': np.mean(scores_lr['test_accuracy']),
    'modelo_std':  np.std(scores_lr['test_accuracy']),
    'AUC_mean': np.mean(scores_lr['test_auc_ovr']),
})

df = pd.DataFrame(resultados)
for c in ['modelo_mean','modelo_std','AUC_mean']:
    df[c] = df[c].round(4)

df_final = df[['modelo','modelo_mean','modelo_std','AUC_mean']].sort_values('modelo_mean', ascending=False)
print(df_final.to_string(index=False))

     modelo  modelo_mean  modelo_std  AUC_mean
 svc_linear       0.9600      0.0533    1.0000
    svc_rbf       0.9600      0.0533    0.9959
LogisticReg       0.9533      0.0670    0.9967
svc_sigmoid       0.8733      0.0696    0.9819
   svc_poly       0.8533      0.1392    0.9417


## Ejercicio 2
- Repite el ejercicio 1 con el dataset `Default`. Utiliza `default` como target.

In [4]:
d = pd.read_csv("Default.csv")
y = d["default"] == "Yes"
Xd = d[['balance', 'income', 'student']]
Xd = pd.get_dummies(Xd, columns=['student'], drop_first=True)
Xd = Xd.astype(np.float64)

In [5]:
indice = list(range(Xd.shape[1]))
preprocessor = ColumnTransformer(
    transformers=[('num', StandardScaler(), indice)],
    remainder='drop')

kfolds = KFold(n_splits=10, shuffle=True, random_state=42)
scoring = {
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc'}

resultados = []

kernels = ['linear', 'poly', 'rbf', 'sigmoid']
for k in kernels:
    svc = SVC(kernel=k, gamma='scale', probability=True, degree=2, random_state=42)

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', svc)
    ])

    scores = cross_validate(pipeline, Xd, y, cv=kfolds, scoring=scoring, n_jobs=-1)
    resultados.append({
        'modelo': f'svc_{k}',
        'modelo_mean': np.mean(scores['test_accuracy']),
        'modelo_std':  np.std(scores['test_accuracy']),
        'AUC_mean': np.mean(scores['test_roc_auc']),
    })

pipe_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])
scores_lr = cross_validate(pipe_lr, Xd, y, cv=kfolds, scoring=scoring, n_jobs=-1)

resultados.append({
    'modelo': 'LogisticReg',
    'modelo_mean': np.mean(scores_lr['test_accuracy']),
    'modelo_std':  np.std(scores_lr['test_accuracy']),
    'AUC_mean': np.mean(scores_lr['test_roc_auc']),
})

df = pd.DataFrame(resultados)
for c in ['modelo_mean','modelo_std','AUC_mean']:
    df[c] = df[c].round(4)

df_final = df[['modelo','modelo_mean','modelo_std','AUC_mean']].sort_values('modelo_mean', ascending=False)
print(df_final.to_string(index=False))

     modelo  modelo_mean  modelo_std  AUC_mean
LogisticReg       0.9733      0.0062    0.9491
    svc_rbf       0.9720      0.0064    0.8394
   svc_poly       0.9692      0.0071    0.8882
 svc_linear       0.9667      0.0067    0.9200
svc_sigmoid       0.9539      0.0077    0.7371


# Addendum

Métricos disponibles para clasificación:
- ‘accuracy’
- ‘balanced_accuracy’
- ‘top_k_accuracy’
- ‘average_precision’
- ‘neg_brier_score’
- ‘f1’
- ‘f1_micro’
- ‘f1_macro’
- ‘f1_weighted’
- ‘f1_samples’
- ‘neg_log_loss’
- ‘precision’ etc.
- ‘recall’ etc.
- ‘jaccard’ etc.
- ‘roc_auc’
- ‘roc_auc_ovr’
- ‘roc_auc_ovo’
- ‘roc_auc_ovr_weighted’
- ‘roc_auc_ovo_weighted’
- ‘d2_log_loss_score’

# References

[1] Shigeo Abe.Support Vector Machines for Pattern Classification,2Ed.Springer-Verlag London,2010. ISBN978-1-84996-097-7. URLhttps://www.springer.com/gp/book/9781849960977.

[2] Johan A K Suykens, Tony Van Gestel, Jos De Brabanter, BartDe Moor, and Joos Vandewalle.Least Squares Support VectorMachines. World Scientific,2002. ISBN9789812381514. URLhttps://www.worldscientific.com/worldscibooks/10.1142/5089.

[3] Bradley, A. P. (1997). The use of the area under the ROC curve in the evaluation of machine learning algorithms. Pattern recognition, 30(7), 1145-1159. URL https://www.researchgate.net/post/how_can_I_interpret_the_ROC_curve_result